# 哈希表

> 哈希表，又叫散列表，是一种基于数组的数据结构，用于快速添加、删除、查找数据

## 原理

> 适用于key-value的数据，key类似于索引，通过哈希函数，把key映射为数组下标，然后把key-value存到对应的位置
>
> 如果该位置已经被其他数据占用，则会发生哈希冲突，而且随着数组中元素越来越多，冲突发生的概率会越来越大
>
> 哈希表设计的关键
>
> 1. 哈希函数：使得数据尽可能的随机、均匀的分配到数组，且本身要足够简单
> 2. 哈希冲突：发生哈希冲突时，数据操作性能可能会下降，要关注、提升最坏的情况下的性能
> 3. 动态扩容：随着数据的增加，为了保持稳定的冲突概率，容量也要跟着上升

### 哈希函数

> 常见的设计方法
> 1. 数据分析法：考虑key的特点、分布，哈希表的大小。例如取手机4位尾号
> 2. 直接寻址法：f(key) = key，要求key范围小，稀疏
> 3. 平方取中法：适合key位数较小
> 4. 折叠法：比如Java中对key的高16位和低16位做异或运算
> 5. 随机数法：

### 哈希冲突

> 常见的解决方法
> 1. 开放寻址法：冲突时在数组上寻找下一个空位，比如线性查找，或者指数查找。优点：可利用CPU缓存，序列化简单；缺点：删除数据需要特殊标记，转载因子不能太大，空间浪费大，扩容代价高
> 2. 拉链法：用链表存储数据。优点：空间利用率高

### 动态扩容

> 常见解决方法
> 1. 一次性扩容：一次性把数据搬迁到新的空间
> 2. 分批次扩容：搬迁操作跟随着其他操作，如新增、查询

## 与链表结合使用

> 哈希表用于快速查找
> 链表用于顺序访问
>
> 场景
> 1. Redis Sorted Set
> 2. Java LinkedHashMap


## 扩展 - Hash算法

> 定义：把不定长的数据映射成定长的数据
>
> 特点
> 1. 单向性
> 2. 对数据变化敏感
> 3. 冲突概率低
> 4. 效率高
>
> 应用场景
> 1. 加密
> 2. 唯一标识
> 3. 数据校验
> 4. hash函数
> 5. 负载均衡
> 6. 数据分片
> 7. 分布式存储

### 一致性Hash算法

> Hash 环是其中一种实现
>
> 核心思想
> 1. 机器的hash值映射到 0 ～ n-1 范围内，n通常是2^64
> 2. 查找数据时，先计算数据hash值（也落在 0 ～ n-1），顺时针寻找离数据点最近的机器

In [ ]:
import bisect
from collections import defaultdict
import random


class ConsistentHash:
    """
    k 虚拟节点个数
    """
    def __init__(self, n, k):
        self.n = n
        self.k = k
        self.vids = set()
        self.machines = {}
        self.data = defaultdict(dict)
    
    def _move_data(self, pre_vid, vid, from_machine_id, to_machine_id):
        for k in self.data[from_machine_id]:
            if k <= vid or vid < pre_vid < k:
                v = self.data[from_machine_id].pop(k)
                self.data[to_machine_id][k] = v

    
    def add_machine(self, id):
        vids = []
        for _ in range(self.k):
            vid = random.randint(0, self.n-1)
            while vid in self.vids:
                vid = random.randint(0, self.n-1)
            self.vids.add(vid)
            vids.append(vid)
            
            # re-balance in real time
            machine_id = self.find_machine(vid)
            if not machine_id or machine_id == id:
                continue
            
            pre_vid = self.find_pre_vid(vid)
            self._move_data(pre_vid, vid, machine_id, id)

        
        self.machines[id] = sorted(vids)
    

    def remove_machine(self, id):
        pass
    
    def find_pre_vid(self, vid):
        pre_vid = None
        distance = self.n
        for key, value in self.machines.items():
            idx = bisect.bisect_right(value, vid)
            if idx == 0 or idx == len(value):
                idx = -1
            d = vid - value[idx]
            if d < 0:
                d += self.n
            
            if d < distance:
                distance = d
                pre_vid = value[idx]
        
        return pre_vid

    
    def find_machine(self, data_key):
        machine_id = None
        distance = self.n
        for key, value in self.machines.items():
            idx = bisect.bisect_left(value, data_key) % len(value)
            d = value[idx] - data_key
            if d < 0:
                d += self.n
            
            if d < distance:
                distance = d
                machine_id = key
        
        return machine_id
    
    def add_data(self, k, v):
        machine_id = self.find_machine(k)
        self.data[machine_id][k] = v
    
    
